In [1]:
seed = 2023
from nlpsig_networks.scripts.swnu_network_functions import (
    obtain_SWNUNetwork_input
)
from nlpsig_networks.utils import obtain_paths_mask, obtain_signatures_mask

In [2]:
import os
os.chdir("nlpsig-networks/notebooks/Rumours/")

## Rumours

In [3]:
%run load_sbert-embeddings.py

In [4]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [5]:
len(df_rumours)

5568

In [6]:
x_data = obtain_SWNUNetwork_input(
    method="umap",
    dimension=3,
    df=df_rumours,
    id_column='timeline_id',
    label_column='label',
    embeddings=sbert_embeddings,
    k=10,
    features=['time_encoding', 'timeline_index'],
    standardise_method=None,
    include_features_in_path=False,
)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


In [7]:
x_data.keys()

dict_keys(['x_data', 'input_channels', 'embedding_dim', 'num_features'])

In [8]:
x_data["x_data"]

{'path': tensor([[[-7.2296,  2.7356,  8.0972],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],
 
         [[-7.2296,  2.7356,  8.0972],
          [-7.4468,  3.6928,  9.9551],
          [ 0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],
 
         [[-7.2296,  2.7356,  8.0972],
          [-7.4468,  3.6928,  9.9551],
          [-9.0931, 12.8440,  9.0442],
          ...,
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],
 
         ...,
 
         [[-7.1256,  2.5965,  9.4014],
          [-5.3229,  6.2228,  9.8129],
          [-5.5247,  3.6294,  9.8226],
          ...,
          [-8.2079,  5.4001, 10.4869],
          [-7.0656,  5.3290, 10.1910],
          

In [9]:
x_data["x_data"]["path"].shape

torch.Size([5568, 10, 3])

In [10]:
x_data["x_data"]["features"]

tensor([[ 6.4823e-02,  9.3748e-03, -3.3364e-02,  ...,  1.8542e-03,
          2.0148e+03,  1.0000e+00],
        [-2.9260e-02, -5.3905e-02, -3.9863e-02,  ...,  9.3606e-02,
          2.0148e+03,  2.0000e+00],
        [-3.1863e-02,  5.5065e-02, -2.4028e-02,  ...,  9.3310e-02,
          2.0148e+03,  3.0000e+00],
        ...,
        [-2.2057e-03,  8.0860e-02, -9.9185e-02,  ..., -2.3929e-02,
          2.0148e+03,  2.2000e+01],
        [-3.1252e-02,  3.5479e-02,  7.2934e-03,  ..., -1.3536e-02,
          2.0148e+03,  2.3000e+01],
        [ 1.1187e-02,  8.0548e-02, -4.1098e-02,  ...,  8.9290e-02,
          2.0148e+03,  2.4000e+01]])

In [11]:
x_data["input_channels"]

3

In [12]:
x_data["embedding_dim"]

384

In [13]:
x_data["num_features"]

2

In [14]:
path = x_data["x_data"]["path"]

In [15]:
path.shape

torch.Size([5568, 10, 3])

In [16]:
import signatory

In [17]:
signatures = signatory.logsignature(path, 2, stream=True)

In [18]:
signatures.shape

torch.Size([5568, 9, 6])

In [19]:
path[0]

tensor([[-7.2296,  2.7356,  8.0972],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)

In [20]:
signatures[0]

tensor([[ 7.2296, -2.7356, -8.0972,  0.0000,  0.0000,  0.0000],
        [ 7.2296, -2.7356, -8.0972,  0.0000,  0.0000,  0.0000],
        [ 7.2296, -2.7356, -8.0972,  0.0000,  0.0000,  0.0000],
        [ 7.2296, -2.7356, -8.0972,  0.0000,  0.0000,  0.0000],
        [ 7.2296, -2.7356, -8.0972,  0.0000,  0.0000,  0.0000],
        [ 7.2296, -2.7356, -8.0972,  0.0000,  0.0000,  0.0000],
        [ 7.2296, -2.7356, -8.0972,  0.0000,  0.0000,  0.0000],
        [ 7.2296, -2.7356, -8.0972,  0.0000,  0.0000,  0.0000],
        [ 7.2296, -2.7356, -8.0972,  0.0000,  0.0000,  0.0000]],
       dtype=torch.float64)

In [21]:
path[2]

tensor([[-7.2296,  2.7356,  8.0972],
        [-7.4468,  3.6928,  9.9551],
        [-9.0931, 12.8440,  9.0442],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)

In [22]:
signatures[2]

tensor([[ -0.2172,   0.9573,   1.8579,   0.0000,   0.0000,   0.0000],
        [ -1.8635,  10.1084,   0.9469,  -0.2056,   1.6283,  -8.9369],
        [  7.2296,  -2.7356,  -8.0972, -34.1965,   5.7501, -48.5668],
        [  7.2296,  -2.7356,  -8.0972, -34.1965,   5.7501, -48.5668],
        [  7.2296,  -2.7356,  -8.0972, -34.1965,   5.7501, -48.5668],
        [  7.2296,  -2.7356,  -8.0972, -34.1965,   5.7501, -48.5668],
        [  7.2296,  -2.7356,  -8.0972, -34.1965,   5.7501, -48.5668],
        [  7.2296,  -2.7356,  -8.0972, -34.1965,   5.7501, -48.5668],
        [  7.2296,  -2.7356,  -8.0972, -34.1965,   5.7501, -48.5668]],
       dtype=torch.float64)

`True` values in the mask are the ones that indicate to MultiheadAttention to ignore in attention calculation.

In [27]:
path_mask = obtain_paths_mask(path)
signatures_mask = obtain_signatures_mask(signatures)

In [36]:
path_mask.shape

torch.Size([5568, 10])

In [37]:
signatures_mask.shape

torch.Size([5568, 9])

In [28]:
path_mask[0]

tensor([False,  True,  True,  True,  True,  True,  True,  True,  True,  True])

In [29]:
signatures_mask[0]

tensor([False,  True,  True,  True,  True,  True,  True,  True,  True])

In [30]:
path_mask_ = (torch.sum(path, 2) == 0)

In [31]:
signatures_mask_ = path_mask[:,:-1]

In [300]:
path_mask_[0]

tensor([False,  True,  True,  True,  True,  True,  True,  True,  True])

In [32]:
torch.equal(path_mask, path_mask_)

True

In [33]:
torch.equal(signatures_mask, signatures_mask_)

True

In [34]:
path_mask.dtype

torch.bool

In [149]:
path[0]

tensor([[ 2.0148e+03,  1.0000e+00, -7.2296e+00,  2.7356e+00,  8.0972e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],
       dtype=torch.float64)

In [150]:
signatures[0]

tensor([[-2.0148e+03, -1.0000e+00,  7.2296e+00, -2.7356e+00, -8.0972e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-2.0148e+03, -1.0000e+00,  7.2296e+00, -2.7356e+00, -8.0972e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-2.0148e+03, -1.0000e+00,  7.2296e+00, -2.7356e+00, -8.0972e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-2.0148e+03, -1.0000e+00,  7.2296e+00, -2.7356e+00, -8.0972e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-2.0148e+03, -1.0000e+00,  7.2296e+00, -2.7356e+00, -8.0972e+00,
          0.0000e+00,  0.0000e+00,

In [151]:
mask[0]

tensor([False,  True,  True,  True,  True,  True,  True,  True,  True])

In [108]:
path[1]

tensor([[ 2.0148e+03,  1.0000e+00, -7.2296e+00,  2.7356e+00,  8.0972e+00],
        [ 2.0148e+03,  2.0000e+00, -7.4468e+00,  3.6928e+00,  9.9551e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],
       dtype=torch.float64)

In [152]:
signatures[1]

tensor([[ 3.0124e-06,  1.0000e+00, -2.1716e-01,  9.5729e-01,  1.8579e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-2.0148e+03, -1.0000e+00,  7.2296e+00, -2.7356e+00, -8.0972e+00,
          1.0074e+03, -2.1876e+02,  9.6438e+02,  1.8716e+03,  3.5062e+00,
         -8.8913e-01, -3.1197e+00, -3.1634e+00, -5.8367e+00, -1.3345e+00],
        [-2.0148e+03, -1.0000e+00,  7.2296e+00, -2.7356e+00, -8.0972e+00,
          1.0074e+03, -2.1876e+02,  9.6438e+02,  1.8716e+03,  3.5062e+00,
         -8.8913e-01, -3.1197e+00, -3.1634e+00, -5.8367e+00, -1.3345e+00],
        [-2.0148e+03, -1.0000e+00,  7.2296e+00, -2.7356e+00, -8.0972e+00,
          1.0074e+03, -2.1876e+02,  9.6438e+02,  1.8716e+03,  3.5062e+00,
         -8.8913e-01, -3.1197e+00, -3.1634e+00, -5.8367e+00, -1.3345e+00],
        [-2.0148e+03, -1.0000e+00,  7.2296e+00, -2.7356e+00, -8.0972e+00,
          1.0074e+03, -2.1876e+02,

In [153]:
mask[1]

tensor([False, False,  True,  True,  True,  True,  True,  True,  True])

## Path mask

In [44]:
import torch.nn as nn
mha = nn.MultiheadAttention(
    embed_dim=path.shape[2],
    num_heads=1,
    batch_first=True,
).double()

In [45]:
attn_output, attn_output_weights = mha(path.double(),
                                       path.double(),
                                       path.double(),
                                       key_padding_mask=path_mask)

In [46]:
i=0
print(path[i])
print(path_mask[i])
print(attn_output[i])

tensor([[-7.2296,  2.7356,  8.0972],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)
tensor([False,  True,  True,  True,  True,  True,  True,  True,  True,  True])
tensor([[ 4.3150, -2.1866,  4.7752],
        [ 4.3150, -2.1866,  4.7752],
        [ 4.3150, -2.1866,  4.7752],
        [ 4.3150, -2.1866,  4.7752],
        [ 4.3150, -2.1866,  4.7752],
        [ 4.3150, -2.1866,  4.7752],
        [ 4.3150, -2.1866,  4.7752],
        [ 4.3150, -2.1866,  4.7752],
        [ 4.3150, -2.1866,  4.7752],
        [ 4.3150, -2.1866,  4.7752]], dtype=torch.float64,
       grad_fn=<SelectBackward>)


In [47]:
i=1
print(path[i])
print(path_mask[i])
print(attn_output[i])

tensor([[-7.2296,  2.7356,  8.0972],
        [-7.4468,  3.6928,  9.9551],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)
tensor([False, False,  True,  True,  True,  True,  True,  True,  True,  True])
tensor([[ 4.4488, -2.4560,  5.1267],
        [ 4.4500, -2.4584,  5.1299],
        [ 4.4136, -2.3851,  5.0342],
        [ 4.4136, -2.3851,  5.0342],
        [ 4.4136, -2.3851,  5.0342],
        [ 4.4136, -2.3851,  5.0342],
        [ 4.4136, -2.3851,  5.0342],
        [ 4.4136, -2.3851,  5.0342],
        [ 4.4136, -2.3851,  5.0342],
        [ 4.4136, -2.3851,  5.0342]], dtype=torch.float64,
       grad_fn=<SelectBackward>)


In [50]:
i=2
print(path[i])
print(path_mask[i])
print(attn_output[i])

tensor([[-7.2296,  2.7356,  8.0972],
        [-7.4468,  3.6928,  9.9551],
        [-9.0931, 12.8440,  9.0442],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)
tensor([False, False, False,  True,  True,  True,  True,  True,  True,  True])
tensor([[ 4.4488, -2.4560,  5.1267],
        [ 4.4500, -2.4584,  5.1299],
        [ 4.3267, -2.2101,  4.8059],
        [ 4.8140, -1.7748,  5.8232],
        [ 4.8140, -1.7748,  5.8232],
        [ 4.8140, -1.7748,  5.8232],
        [ 4.8140, -1.7748,  5.8232],
        [ 4.8140, -1.7748,  5.8232],
        [ 4.8140, -1.7748,  5.8232],
        [ 4.8140, -1.7748,  5.8232]], dtype=torch.float64,
       grad_fn=<SelectBackward>)


In [51]:
i=4
print(path[i])
print(path_mask[i])
print(attn_output[i])

tensor([[-7.2296,  2.7356,  8.0972],
        [-7.4468,  3.6928,  9.9551],
        [-9.0931, 12.8440,  9.0442],
        [-6.7662,  2.6486,  8.3875],
        [-5.3710,  7.5870, 12.4986],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)
tensor([False, False, False, False, False,  True,  True,  True,  True,  True])
tensor([[ 4.3171, -2.4032,  4.9670],
        [ 4.3217, -2.4011,  4.9583],
        [ 4.1344, -2.2620,  4.6543],
        [ 4.3273, -2.4105,  4.9804],
        [ 4.2305, -2.3234,  4.8004],
        [ 4.4062, -2.0321,  5.4916],
        [ 4.4062, -2.0321,  5.4916],
        [ 4.4062, -2.0321,  5.4916],
        [ 4.4062, -2.0321,  5.4916],
        [ 4.4062, -2.0321,  5.4916]], dtype=torch.float64,
       grad_fn=<SelectBackward>)


## Signatures mask

In [52]:
import torch.nn as nn
mha = nn.MultiheadAttention(
    embed_dim=signatures.shape[2],
    num_heads=1,
    batch_first=True,
).double()

In [53]:
attn_output, attn_output_weights = mha(signatures.double(),
                                       signatures.double(),
                                       signatures.double(),
                                       key_padding_mask=signatures_mask)

In [58]:
i=0
print(path[i])
print(signatures[i])
print(signatures_mask[i])
print(attn_output[i])

tensor([[-7.2296,  2.7356,  8.0972],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)
tensor([[ 7.2296, -2.7356, -8.0972,  0.0000,  0.0000,  0.0000],
        [ 7.2296, -2.7356, -8.0972,  0.0000,  0.0000,  0.0000],
        [ 7.2296, -2.7356, -8.0972,  0.0000,  0.0000,  0.0000],
        [ 7.2296, -2.7356, -8.0972,  0.0000,  0.0000,  0.0000],
        [ 7.2296, -2.7356, -8.0972,  0.0000,  0.0000,  0.0000],
        [ 7.2296, -2.7356, -8.0972,  0.0000,  0.0000,  0.0000],
        [ 7.2296, -2.7356, -8.0972,  0.0000,  0.0000,  0.0000],
        [ 7.2296, -2.7356, -8.0972,  0.0000,  0.0000,  0.0000],
        [ 7.2296, -2.7356, -8.0972,  0.0000,  0.0000,  0.0000]],
       dtype=torch.float64)
ten

In [59]:
i=1
print(path[i])
print(signatures[i])
print(signatures_mask[i])
print(attn_output[i])

tensor([[-7.2296,  2.7356,  8.0972],
        [-7.4468,  3.6928,  9.9551],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)
tensor([[-0.2172,  0.9573,  1.8579,  0.0000,  0.0000,  0.0000],
        [ 7.2296, -2.7356, -8.0972, -3.1634, -5.8367, -1.3345],
        [ 7.2296, -2.7356, -8.0972, -3.1634, -5.8367, -1.3345],
        [ 7.2296, -2.7356, -8.0972, -3.1634, -5.8367, -1.3345],
        [ 7.2296, -2.7356, -8.0972, -3.1634, -5.8367, -1.3345],
        [ 7.2296, -2.7356, -8.0972, -3.1634, -5.8367, -1.3345],
        [ 7.2296, -2.7356, -8.0972, -3.1634, -5.8367, -1.3345],
        [ 7.2296, -2.7356, -8.0972, -3.1634, -5.8367, -1.3345],
        [ 7.2296, -2.7356, -8.0972, -3.1634, -5.8367, -1.3345]],
       dtype=torch.float64)
ten

In [60]:
i=2
print(path[i])
print(signatures[i])
print(signatures_mask[i])
print(attn_output[i])

tensor([[-7.2296,  2.7356,  8.0972],
        [-7.4468,  3.6928,  9.9551],
        [-9.0931, 12.8440,  9.0442],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)
tensor([[ -0.2172,   0.9573,   1.8579,   0.0000,   0.0000,   0.0000],
        [ -1.8635,  10.1084,   0.9469,  -0.2056,   1.6283,  -8.9369],
        [  7.2296,  -2.7356,  -8.0972, -34.1965,   5.7501, -48.5668],
        [  7.2296,  -2.7356,  -8.0972, -34.1965,   5.7501, -48.5668],
        [  7.2296,  -2.7356,  -8.0972, -34.1965,   5.7501, -48.5668],
        [  7.2296,  -2.7356,  -8.0972, -34.1965,   5.7501, -48.5668],
        [  7.2296,  -2.7356,  -8.0972, -34.1965,   5.7501, -48.5668],
        [  7.2296,  -2.7356,  -8.0972, -34.1965,   5.7501, -48.5668],
        [  7.2296,  -2.7356,  -8.0972, -34.1965,

In [61]:
i=4
print(path[i])
print(signatures[i])
print(signatures_mask[i])
print(attn_output[i])

tensor([[-7.2296,  2.7356,  8.0972],
        [-7.4468,  3.6928,  9.9551],
        [-9.0931, 12.8440,  9.0442],
        [-6.7662,  2.6486,  8.3875],
        [-5.3710,  7.5870, 12.4986],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)
tensor([[ -0.2172,   0.9573,   1.8579,   0.0000,   0.0000,   0.0000],
        [ -1.8635,  10.1084,   0.9469,  -0.2056,   1.6283,  -8.9369],
        [  0.4634,  -0.0869,   0.2903,  -2.4666,   1.1384,  -7.4285],
        [  1.8586,   4.8514,   4.4014,  -1.2618,   1.8884,  -8.3240],
        [  7.2296,  -2.7356,  -8.0972, -21.3409, -21.5464, -21.9455],
        [  7.2296,  -2.7356,  -8.0972, -21.3409, -21.5464, -21.9455],
        [  7.2296,  -2.7356,  -8.0972, -21.3409, -21.5464, -21.9455],
        [  7.2296,  -2.7356,  -8.0972, -21.3409, -21.5464, -21.9455],
        [  7.2296,  -2.7356,  -8.0972, -21.3409,